## Transfer Scene Viewing Eyetracking Files From USB to Computer

This script will keep a log of when it was last synced. A prompt will appear asking for confirmation of the files that you would like to push to the computer. It is designed so that any files created after the last sync will be copied over (meaning you could rename a file and they won't be copied over again). This script will push the session folders from each block to their corresponding block folders on the computer. 

VERSION: `Scene_Eyetrack_Auto_Push`

In [ ]:
$source = "E:\SceneViewing_DAY3" 
$destination = "C:\Backupdata\SceneViewing_DAY3" 
$logFile = "C:\Backupdata\SceneViewing_DAY3\DAY3_USB_Log.txt" 

if (-not (Test-Path -Path $logFile)) {
    New-Item -Path $logFile -ItemType File
    "Log Created: $(Get-Date)" | Out-File -FilePath $logFile
}

$lastSyncTime = (Get-Content -Path $logFile -Tail 1) -replace "Last Sync:", ""
if ([string]::IsNullOrWhiteSpace($lastSyncTime)) {
    $lastSyncTime = [datetime]::MinValue
} else {
    $lastSyncTime = [datetime]::Parse($lastSyncTime)
}

$blockTypes = @("Encoding", "Retrieval")
$blockNumbers = 1..4
$noNewFolders = $true

foreach ($type in $blockTypes) {
    foreach ($number in $blockNumbers) {
        $blockName = "${type}_Block$number"
        $sourcePath = Join-Path -Path $source -ChildPath $blockName
        $destinationPath = Join-Path -Path $destination -ChildPath $blockName

        if (-not (Test-Path -Path $sourcePath)) {
            Write-Output "Source path $sourcePath does not exist."
            continue
        }

        if (-not (Test-Path -Path $destinationPath)) {
            New-Item -Path $destinationPath -ItemType Directory
        }

        $newData = Get-ChildItem -Path $sourcePath -Directory | Where-Object { $_.CreationTime -gt $lastSyncTime }

        if ($newData.Count -eq 0) {
            Write-Output "No new session folders in $blockName since last synced."
            continue
        }

        $noNewFolders = $false

        foreach ($dir in $newData) {
            $dirSizeBytes = (Get-ChildItem -Path $dir.FullName -Recurse -File | Measure-Object -Property Length -Sum).Sum
            $dirSizeMB = [math]::Round($dirSizeBytes / 1MB, 2)

            $userInput = Read-Host "Do you want to push $($dir.Name) [Size: $($dirSizeMB) MB] [Y/N]"

            if ($userInput -eq "Y") {
                $destPath = Join-Path -Path $destinationPath -ChildPath $dir.Name
                Copy-Item -Path $dir.FullName -Destination $destPath -Recurse
                Write-Output "Copied $($dir.Name) to $destinationPath"
            }
        }
    }
}

if ($noNewFolders) {
    Write-Output "No new session folders across all blocks since last synced."
}

"Last Sync: $(Get-Date)" | Out-File -FilePath $logFile -Append

Read-Host -Prompt "Press Enter to Close"